In [ ]:
import os
from glob import glob
from PIL import Image

In [ ]:
import torchvision.transforms as transforms

transform=transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])  

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch

class PlanetDiseaseDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = glob(os.path.join(root_dir, '*', '*.jpg'))
        self.class_names = sorted(os.listdir(root_dir))
        self.class_to_idx = {class_name: idx for idx, class_name in enumerate(self.class_names)}

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        class_name = os.path.basename(os.path.dirname(image_path))
        label = self.class_to_idx[class_name]

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
train_dataset = PlanetDiseaseDataset(root_dir='data/plant_diseases/train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

valid_dataset = PlanetDiseaseDataset(root_dir='data/plant_diseases/valid', transform=transform)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 32 * 32, 512)
        self.fc2 = nn.Linear(512, num_classes)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 32 * 32)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
num_classes = len(train_dataset.class_names)
model = SimpleCNN(num_classes=num_classes)

In [ ]:
epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

from tqdm import tqdm

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader):
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    avg_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}')
    
    model.eval()
    correct = 0
    
    total = 0
    
    with torch.no_grad():
        for images, labels in tqdm(valid_loader):
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Validation Accuracy: {accuracy:.2f}%')

In [ ]:
def predict_image(image_path, model, transform, class_names):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)  # Add batch dimension
    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs.data, 1)
    return class_names[predicted.item()]

In [ ]:
test_images="/kaggle/input/new-plant-diseases-dataset/test/test"

for img_file in glob(os.path.join(test_images, '*.jpg')):
    prediction = predict_image(img_file, model, transform, train_dataset.class_names)
    print(f'Image: {os.path.basename(img_file)}, Predicted Class: {prediction}')